# ML Script

In [ ]:
run_in = 'colab'
# run_in = 'local'
# run_in = 'colab&gsheets'

if run_in == 'local':
    import os
    import math
    from collections import Counter
    import numpy as np
    import seaborn as sns
    import pandas as pd
    import scipy.stats as ss
    import matplotlib.pyplot as plt
    !pip install scikit-learn
    import sklearn.preprocessing as sp
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    from subprocess import check_output

    from sklearn import svm, datasets
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import label_binarize
    from sklearn.multiclass import OneVsRestClassifier
    from dython.nominal import conditional_entropy
    from dython.nominal import theils_u
    from dython.nominal import associations

    from sklearn.metrics import make_scorer
    from sklearn.model_selection import GridSearchCV

    from sklearn.metrics import r2_score

    # from sklearn.preprocessing import LabelEncoder
    # labelencoder = LabelEncoder()

    import itertools

    class color:
      PURPLE = '\033[95m'
      CYAN = '\033[96m'
      DARKCYAN = '\033[36m'
      BLUE = '\033[94m'
      GREEN = '\033[92m'
      YELLOW = '\033[93m'
      RED = '\033[91m'
      BOLD = '\033[1m'
      UNDERLINE = '\033[4m'
      END = '\033[0m'

    import pprint
    pp = pprint.PrettyPrinter(indent=4)

    #for CramersV
    from dython._private import (convert, remove_incomplete_samples, replace_nan_with_value)
    REPLACE = 'replace'
    DROP = 'drop'
    DROP_SAMPLES = 'drop_samples'
    DROP_FEATURES = 'drop_features'
    SKIP = 'skip'
    DEFAULT_REPLACE_VALUE = 0.0
    
if run_in == 'colab' or run_in == 'colab&gsheets':  
    import os
    import math
    from collections import Counter
    import numpy as np
    import seaborn as sns
    import pandas as pd
    import scipy.stats as ss
    import matplotlib.pyplot as plt
    !pip install scikit-learn
    # https://github.com/shakedzy/dython/blob/master/dython/nominal.py
    !pip install dython
    from dython.nominal import conditional_entropy
    from dython.nominal import theils_u
    from dython.nominal import associations
    
    import sklearn.preprocessing as sp
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    from subprocess import check_output

    from sklearn import svm, datasets
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import label_binarize
    from sklearn.multiclass import OneVsRestClassifier

    from sklearn.metrics import make_scorer
    from sklearn.model_selection import GridSearchCV

    from sklearn.metrics import r2_score

    # from sklearn.preprocessing import LabelEncoder
    # labelencoder = LabelEncoder()

    import itertools
    
    class color:
      PURPLE = '\033[95m'
      CYAN = '\033[96m'
      DARKCYAN = '\033[36m'
      BLUE = '\033[94m'
      GREEN = '\033[92m'
      YELLOW = '\033[93m'
      RED = '\033[91m'
      BOLD = '\033[1m'
      UNDERLINE = '\033[4m'
      END = '\033[0m'

    import pprint
    pp = pprint.PrettyPrinter(indent=4)

    #for CramersV
    from dython._private import (convert, remove_incomplete_samples, replace_nan_with_value)
    REPLACE = 'replace'
    DROP = 'drop'
    DROP_SAMPLES = 'drop_samples'
    DROP_FEATURES = 'drop_features'
    SKIP = 'skip'
    DEFAULT_REPLACE_VALUE = 0.0

In [ ]:
if run_in == 'local':
 base_path = '/Users/timbeck/Downloads/'

if run_in == 'colab':  
  base_path = '/content/drive/My Drive/USERNAME/Analysis' # for google cloud
  # base_path = '/content/drive/My Drive/USERNAMEMLattempt/' # for google cloud USERNAME
  from google.colab import drive
  drive.mount('/content/drive')

if run_in == 'colab&gsheets':  
  base_path = '/content/drive/My Drive/Colab Notebooks/' # for google sheets
  from google.colab import auth
  auth.authenticate_user()
  import gspread
  from oauth2client.client import GoogleCredentials
  gc = gspread.authorize(GoogleCredentials.get_application_default())

print('base_path set to:')
print(base_path)

In [ ]:
os.listdir(base_path)

# Functions

In [ ]:
def cramers_v(x, y, nan_strategy=REPLACE, nan_replace_value=DEFAULT_REPLACE_VALUE):
    """
    Calculates Cramer's V statistic for categorical-categorical association.
    Uses correction from Bergsma and Wicher, Journal of the Korean Statistical
    Society 42 (2013): 323-328.
    This is a symmetric coefficient: V(x,y) = V(y,x)
    Original function taken from: https://stackoverflow.com/a/46498792/5863503
    Wikipedia: https://en.wikipedia.org/wiki/Cram%C3%A9r%27s_V
    **Returns:** float in the range of [0,1]
    Parameters
    ----------
    x : list / NumPy ndarray / Pandas Series
        A sequence of categorical measurements
    y : list / NumPy ndarray / Pandas Series
        A sequence of categorical measurements
    nan_strategy : string, default = 'replace'
        How to handle missing values: can be either 'drop' to remove samples
        with missing values, or 'replace' to replace all missing values with
        the nan_replace_value. Missing values are None and np.nan.
    nan_replace_value : any, default = 0.0
        The value used to replace missing values with. Only applicable when
        nan_strategy is set to 'replace'.
    """

    # print(x)
    # print(y)
    # print(nan_strategy)
    # print(nan_replace_value)

    if nan_strategy == REPLACE:
        x, y = replace_nan_with_value(x, y, nan_replace_value)

        # print("NEW X AND Y")
        # print(x)
        # print("lenx", len(x))
        # print(y)
        # print("leny", len(y))

    elif nan_strategy == DROP:
        x, y = remove_incomplete_samples(x, y)

        # print("DROP NAN X AND Y")
        # print("type x", type(x))
        # print(x)
        # print("lenx", len(x))

        # print(y)
        # print("type y", type(y))
        # print("leny", len(y))
    
    ax = np.asarray(x)
    ay = np.asarray(y)

    confusion_matrix = pd.crosstab(ax, ay)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    # print("chi2=", chi2)
    n = confusion_matrix.sum().sum()
    # print("n",n)
    phi2 = chi2 / n
    # print("phi2=", phi2)
    r, k = confusion_matrix.shape
    # print("r=", r)
    # print("k=", k)
    phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
    rcorr = r - ((r - 1)**2) / (n - 1)
    kcorr = k - ((k - 1)**2) / (n - 1)
    # print("rcorr=", rcorr)
    # print("kcorr=", kcorr)
    c_v = np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

    #pvalue p_v, with degrees of freedom (rcorr-1)*(kcorr-1)
    dof = (rcorr-1)*(kcorr-1)
    # print("dof", dof)
    p_v = 1 - ss.chi2.cdf(chi2, dof)
    
    # print("c_v=", c_v)
    # print("p_v=", p_v)

    return c_v, p_v

In [ ]:
def theils_u(x, y, nan_strategy=REPLACE, nan_replace_value=DEFAULT_REPLACE_VALUE):
    """
    Calculates Theil's U statistic (Uncertainty coefficient) for categorical-
    categorical association. This is the uncertainty of x given y: value is
    on the range of [0,1] - where 0 means y provides no information about
    x, and 1 means y provides full information about x.
    This is an asymmetric coefficient: U(x,y) != U(y,x)
    Wikipedia: https://en.wikipedia.org/wiki/Uncertainty_coefficient
    **Returns:** float in the range of [0,1]
    Parameters
    ----------
    x : list / NumPy ndarray / Pandas Series
        A sequence of categorical measurements
    y : list / NumPy ndarray / Pandas Series
        A sequence of categorical measurements
    nan_strategy : string, default = 'replace'
        How to handle missing values: can be either 'drop' to remove samples
        with missing values, or 'replace' to replace all missing values with
        the nan_replace_value. Missing values are None and np.nan.
    nan_replace_value : any, default = 0.0
        The value used to replace missing values with. Only applicable when
        nan_strategy is set to 'replace'.
    """
    if nan_strategy == REPLACE:
        x, y = replace_nan_with_value(x, y, nan_replace_value)
    elif nan_strategy == DROP:
        x, y = remove_incomplete_samples(x, y)
    s_xy = conditional_entropy(x, y)
    # print(s_xy)
    x_counter = Counter(x)
    total_occurrences = sum(x_counter.values())
    p_x = list(map(lambda n: n / total_occurrences, x_counter.values()))
    s_x = ss.entropy(p_x)
    if s_x == 0:
        t_u = 1
    else:
        t_u = (s_x - s_xy) / s_x
    return t_u

In [ ]:
def interval_documentor(variable,cut_task,bin_names):
  variable_intervals = []
  number_of_bins = len(cut_task.dtype.categories)
  # print("number_of_bins",number_of_bins)
  for i in range(0,number_of_bins):
    # print("list_index",i)
    str_i = str(list(cut_task.dtype.categories)[i])
    variable_intervals.append(str_i)
  zip_variable_intervals = list(zip(variable_intervals,bin_names))
  return zip_variable_intervals

In [ ]:
def half_split(df,x_header_select):
  #cuts the data at 0 and makes two groups on each side at 0.5 quantile
  df_temp_low = df[df[x_header_select]<0]
  df_temp_high = df[df[x_header_select]>=0]
  lowest = df_temp_low[x_header_select].min()-1
  low_mid = df_temp_low[x_header_select].quantile(0.5)
  high_mid = df_temp_high[x_header_select].quantile(0.5)
  highest = df_temp_high[x_header_select].max()+1
  smaller_than_zero = -0.000000001
  generated_bins = [lowest, low_mid, smaller_than_zero, 0, high_mid, highest]
  return generated_bins

In [ ]:
def zero_thirds(df,x_header_select):
  #cuts the 0 off and the rest in thirds
  df_no_zero = df[df[x_header_select]!=0]
  low = df_no_zero[x_header_select].quantile(0.33)
  mid = df_no_zero[x_header_select].quantile(0.66)
  high = df_no_zero[x_header_select].max()+1
  smaller_than_zero = -0.000000001
  generated_bins = [smaller_than_zero, 0, low, mid, high]
  return generated_bins

In [ ]:
def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    
    # TODO: Calculate the performance score between 'y_true' and 'y_predict'
    score = r2_score(y_true, y_predict)
    
    # Return the score
    return score

In [ ]:
def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.20, random_state = 0)

    # Create a decision tree regressor object
    regressor = DecisionTreeRegressor()

    # Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {'max_depth':[1,2,3,4,5,6,7,8,9,10]}

    # Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(performance_metric)

    # Create the grid search cv object --> GridSearchCV()
    # Make sure to include the right parameters in the object:
    # (estimator, param_grid, scoring, cv) which have values 'regressor', 'params', 'scoring_fnc', and 'cv_sets' respectively.
    grid = GridSearchCV(estimator=regressor, param_grid=params, scoring=scoring_fnc, cv=cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

In [ ]:
def cramer_df_selector(df_selected_variable,df_selected_target,save = True):
  selected_variables = list(df_selected_variable.columns)
  selected_target = list(df_selected_variable.columns)

  col_combinations = list(itertools.product(df_selected_variable.columns,df_selected_target.columns))

  variable_list = []
  target_list = []
  c_v_list = []
  p_v_list = []
  t_u_list = []

  for i in col_combinations:
    x = df_variables[i[0]]
    y = df_targets[i[1]]

    c_v, p_v = cramers_v(x, y, nan_strategy = REPLACE, nan_replace_value = DEFAULT_REPLACE_VALUE)
    variable = i[0]
    target = i[1]

    variable_list.append(variable)
    target_list.append(target)
    c_v_list.append(c_v)
    p_v_list.append(p_v)

    t_u = theils_u(x, y, nan_strategy = REPLACE, nan_replace_value = DEFAULT_REPLACE_VALUE)
    t_u_list.append(t_u)

  df_result = pd.DataFrame()
  df_result["variable"] = variable_list
  df_result["target"] = target_list
  df_result["c_v"] = c_v_list
  df_result["p_v"] = p_v_list
  df_result["t_u"] = t_u_list

  # sort out all the c_v values based on the p-value p_v:
  df_result_p_vrelevant = df_result[df_result["p_v"]<=0.1]
  df_result_p_vrelevant = df_result_p_vrelevant.reset_index(drop=True)
  p_v_relevant_headers = [h for h in list(df.columns) if h in list(set(df_result_p_vrelevant["variable"]))]

  # sort out all the c_v values based on the association relevance threshold [weak assocation if c_v>0.1]:
  df_result_val_and_p_vrelevant = df_result_p_vrelevant[df_result_p_vrelevant["c_v"]>=0.1]
  df_result_val_and_p_vrelevant = df_result_val_and_p_vrelevant.reset_index(drop=True)
  val_and_p_v_relevant_headers = [h for h in list(df.columns) if h in list(set(df_result_val_and_p_vrelevant["variable"]))]

  if save:
  # save to excel
    with pd.ExcelWriter(os.path.join(base_path,'Twitter_Cramer_Theil_.xlsx')) as writer:
      df_result.to_excel(writer, sheet_name= "Cramer_V_and_Theil_U", index=True) 
      df_result_p_vrelevant.to_excel(writer, sheet_name= "Rel_C_V_and_T_U", index=True) 
      df_result_val_and_p_vrelevant.to_excel(writer, sheet_name= "Rel_val_p_v_C_V_and_T_U", index=True) 

  # create the dataframe comprising the determined relevant variables and the selected target:
  df_c_v_and_p_v_relevant = df[val_and_p_v_relevant_headers+selected_target]

  print(val_and_p_v_relevant_headers)
  print(len(val_and_p_v_relevant_headers))

  return val_and_p_v_relevant_headers

In [ ]:
def ML(df,ML_selected_variables,ML_selected_target,no_iterations):
  '''
  variables must be a lists
  target must be a string
  '''
  factorized_nw = df[selected_variables+[selected_target]].copy()
  for column in factorized_nw.columns.values:
      f, _ = pd.factorize(factorized_nw[column])
      factorized_nw.loc[:,column] = f
  ohe = sp.OneHotEncoder()
  X = factorized_nw.drop([selected_target],axis=1)
  y = factorized_nw[selected_target].tolist()
  ohe.fit(X)
  X = ohe.transform(X).toarray()
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

  print("selected_target:", selected_target)
  for i in range(1,no_iterations+1):
      tree = DecisionTreeClassifier(splitter='best', max_depth=i, random_state=42)
      tree.fit(X_train,y_train)
      y_pred = tree.predict(X_test)
      
      print("Max depth: {} - accuracy:".format(i), accuracy_score(y_test, y_pred, normalize=True))
      print()

# Create categories

In [ ]:
if run_in == 'colab' or run_in == 'local':
  df = pd.read_excel(os.path.join(base_path,"Data_Prep_Database.xlsx"), skiprows = 0)

df = df.rename(index=str, columns={"Series A" : "Series_A"})
df = df.rename(index=str, columns={"Series B" : "Series_B"})

#exclude columns:
selected_target_list = ['dt_avg', 'num_sum_raised_to_date', 'max_post_money_valuation']
exclude_sheet_name_variables = ['company_name','founded_date',"num_company_age_to_today",'London',  'Cambridge', 'Oxford', 'Bristol', 'Other', "num_log10_sum_raised_to_date", "log10_max_post_money_valuation",'last_stage_before_survey','last_stage_during_survey','fundraising_cadence','fundraising_cadence_category']
sheet_name_variables = [h for h in list(df.columns) if h not in selected_target_list and h not in exclude_sheet_name_variables]
df = df[sheet_name_variables + selected_target_list]

fill_nan = True
give_category_names = True

all_variable_interval_description_dict = {}

################## To TEST STUFF ##################
# df[["dt_avg"]]
print(df["dt_avg"].describe())
###################################################

counter = 0
for x_header in list(df.columns):
  counter +=1

  # ########## TARGETS ############################################################################
  # #-----------------------------------------------------------------------------------------------------------------#

  # if x_header == "dt_avg":
  #   #decide if the final categories should be named or numerical categories:
  #   df[x_header] = df[x_header].fillna(-10e6)
  #   if give_category_names:
  #     bin_labels = ["X1_NA","1_very_short_dt_avg","2_short_dt_avg","3_long_dt_avg", "4_very_long_dt_avg"]
  #   if not give_category_names:
  #     bin_labels = quattro
  #   #define how we want to cut
  #   cut_type = "cut"    #cuts with a self defined list
  #   # cut_type = "qcut" #cuts into q=n n different buckets
  
  #   if cut_type == "qcut": 
  #     #document cutting:
  #     zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
  #     #cut and overwrite:
  #     if fill_nan:
  #       df[x_header] = df[x_header].fillna(10e4)
  #     df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
  #   if cut_type == "cut":
  #     cut_info = ['Slicing info: Self defined thresholds'] 
  #     #document cutting:
  #     bin_values = [-11e6,-1e-09, 0, 243.3308333333333, 406.43, 1750.0]
  #     zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
  #     #cut and overwrite:
  #     # if fill_nan:
  #       # df[x_header] = df[x_header].fillna(10e4)
  #     df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
  #   all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  # #-----------------------------------------------------------------------------------------------------------------#

In [ ]:
if run_in == 'colab' or run_in == 'local':
  df = pd.read_excel(os.path.join(base_path,"Data_Prep_Database.xlsx"), skiprows = 0)

df = df.rename(index=str, columns={"Series A" : "Series_A"})
df = df.rename(index=str, columns={"Series B" : "Series_B"})

#exclude columns:
selected_target_list = ['dt_avg', 'num_sum_raised_to_date', 'max_post_money_valuation']
exclude_sheet_name_variables = ['company_name','founded_date',"num_company_age_to_today",'London',  'Cambridge', 'Oxford', 'Bristol', 'Other', "num_log10_sum_raised_to_date", "log10_max_post_money_valuation",'last_stage_before_survey','last_stage_during_survey','fundraising_cadence','fundraising_cadence_category']
fnf_sna = [h for h in df.columns if "fnf" in h]
fr_weighted_sna = [h for h in df.columns if "fr_weighted" in h]

sheet_name_variables = [h for h in list(df.columns) if h not in selected_target_list and h not in exclude_sheet_name_variables and h not in fnf_sna and h not in fr_weighted_sna]
df = df[sheet_name_variables + selected_target_list]

fill_nan = True
give_category_names = True

all_variable_interval_description_dict = {}

################## To TEST STUFF ##################
# # df[["fnf_degree_mean"]]
# print(df["fnf_degree_mean"].describe())
###################################################

counter = 0
for x_header in list(df.columns):
  counter +=1

  ########## TARGETS ############################################################################
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "dt_avg":
    #decide if the final categories should be named or numerical categories:
    df[x_header] = df[x_header].fillna(-10e6)
    if give_category_names:
      bin_labels = ["X1_NA","1_pre_inc_dt_avg","2_short_dt_avg","3_medium_dt_avg", "4_long_dt_avg"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut"    #cuts with a self defined list
    # cut_type = "qcut" #cuts into q=n n different buckets
  
    if cut_type == "qcut": 
      #document cutting:
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(10e4)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Self defined thresholds'] 
      #document cutting:
      bin_values = [-11e6,-3, 0, 205, 315, 10e6]
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(10e4)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "num_sum_raised_to_date":
    #decide if the final categories should be named or numerical categories:
    df[x_header] = df[x_header].fillna(0)
    df[x_header] = df[x_header].replace(0, 1)
    df[x_header] = np.log10(df[x_header])
    if give_category_names:
      bin_labels = ["1_no_funding", "2_five_figure", "3_half_mil", "4_six_figure","5_five_mil", "6_seven_figure", "7_eight_figure_plus"]
    if not give_category_names:
      bin_labels = [1,2,3,4,5,6,7]
    #define how we want to cut
    cut_type = "cut"    #cuts with a self defined list
    # cut_type = "qcut" #cuts into q=n n different buckets

    if cut_type == "qcut": 
      pass
      #document cutting:
      # zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Self defined thresholds'] 
      #document cutting:
      bin_values = [-0.001,5,5.69897000434,6,6.69897000434,7,8,10]
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        print("fillna happens")
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "max_post_money_valuation":
    #decide if the final categories should be named or numerical categories:
    df[x_header] = df[x_header].replace(0, 1)
    df[x_header] = np.log10(df[x_header])
    if give_category_names:
      bin_labels = ["1_no_valuation", "2_five_figure", "3_half_mil", "4_six_figure","5_five_mil", "6_seven_figure", "7_eight_figure_plus"]
    if not give_category_names:
      bin_labels = [1,2,3,4,5,6,7]
    #define how we want to cut
    cut_type = "cut"    #cuts with a self defined list
    # cut_type = "qcut" #cuts into q=n n different buckets

    if cut_type == "qcut": 
      pass
      #document cutting:
      # zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Self defined thresholds'] 
      #document cutting:
      bin_values = [-0.001,5,5.69897000434,6,6.69897000434,7,8,10]
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  ########## VARIABLES ############################################################################
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "num_company_age_to_today":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_young_company_age","2_young_company_age","3_mature_company_age", "4_very_mature_company_age"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    # cut_type = "cut" #cuts with a self defined list
    cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut":
      cut_info = ['Slicing info: Qartile cut']
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut": 
      #document cutting:
      bin_values = []
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
#-----------------------------------------------------------------------------------------------------------------#

  if x_header == "inbound_links":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_no_inbound_links","2_little_inbound_links","3_medium_inbound_links", "4_high_inbound_links"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    # cut_type = "cut" #cuts with a self defined list
    cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut":
      cut_info = ['Slicing info: Qartile cut']
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut": 
      #document cutting:
      bin_values = []
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "global_pagerank":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_good_global_pagerank","2_good_global_pagerank","3_bad_global_pagerank", "4_very_bad_global_pagerank", "5_no_global_pagerank"]
    if not give_category_names:
      bin_labels = [1,2,3,4,5]
    #define how we want to cut
    cut_type = "cut"    #cuts with a self defined list
    # cut_type = "qcut" #cuts into q=n n different buckets
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(2e7)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Self defined thresholds'] 
      #document cutting:
      bin_values = [10047.999,2220568,4461287,8317489,19544569,2e7]
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(2e7)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "has_twitter":
    df[x_header] = df[x_header].replace(1,True)
    df[x_header] = df[x_header].replace(0,False)

  if x_header == "has_website":
    df[x_header] = df[x_header].replace(1,True)
    df[x_header] = df[x_header].replace(0,False)

  if x_header == "b2b":
    df[x_header] = df[x_header].replace(1,True)
    df[x_header] = df[x_header].replace(0,False)

  if x_header == "had_event":
    df[x_header] = df[x_header].replace(1,True)
    df[x_header] = df[x_header].replace(0,False)
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "tweet_count":
      #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low_num_tweets","2_medium_num_tweets", "4_high_num_tweets"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
  
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "tweets_per_day":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low_num_tweets","2_medium_num_tweets", "4_high_num_tweets"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
  
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "last_stage_before_survey":
    print(x_header, "is already categorical")

  if x_header == "last_stage_during_survey":
    print(x_header, "is already categorical")

  if x_header == "fundraising_cadence":
    print(x_header, "is already categorical")

  if x_header == "fundraising_cadence_category":
    print(x_header, "is already categorical")

  if x_header == "Angel":
    df[x_header] = df[x_header].replace(1,True)
    df[x_header] = df[x_header].replace(0,False)
  
  if x_header == "Grant":
    df[x_header] = df[x_header].replace(1,True)
    df[x_header] = df[x_header].replace(0,False)
  
  if x_header == "Crowdfunding":
    df[x_header] = df[x_header].replace(1,True)
    df[x_header] = df[x_header].replace(0,False)

  if x_header == "Seed":
    df[x_header] = df[x_header].replace(1,True)
    df[x_header] = df[x_header].replace(0,False)
  
  if x_header == "Series_A":
    df[x_header] = df[x_header].replace(1,True)
    df[x_header] = df[x_header].replace(0,False)

  if x_header == "Series_B":
    df[x_header] = df[x_header].replace(1,True)
    df[x_header] = df[x_header].replace(0,False)
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "hq_city_ecosystem":
    print(x_header, "is already categorical")  
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "num_sum_investors":
    #decide if the final categories should be named or numerical categories:
    df[x_header] = df[x_header].fillna(0)
    if give_category_names:
      bin_labels = ["1_no_investors","2_low_num_investors", "3_medium_num_investors", "4_high_num_investors"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#
    
  if x_header == "num_sum_follow_on_investors":
    #decide if the final categories should be named or numerical categories:
    df[x_header] = df[x_header].fillna(0)
    if give_category_names:
      bin_labels = ["1_no_foll_investors","2_low_num_foll_investors", "3_medium_num_foll_investors", "4_high_num_foll_investors"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "accelerated":
    df[x_header] = df[x_header].replace(1,True)
    df[x_header] = df[x_header].replace(0,False)

  if x_header == "invested":
    df[x_header] = df[x_header].replace(1,True)
    df[x_header] = df[x_header].replace(0,False)
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "new_to_followon_investor_ratio":
    #decide if the final categories should be named or numerical categories:
    if fill_nan:
      df[x_header] = df[x_header].fillna(-1)
    bin_labels = ["X1_NA", "X2_No_investors", "1_no_follow_on","2_low_follow_on", "3_medium_follow_on", "4_high_follow_on"]
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    
    if cut_type == "cut": 
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = [-2,-0.0001,0,2,3.75,5,20]
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(10e7)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#
      
  if x_header == "num_rounds":
    #decide if the final categories should be named or numerical categories:
    df[x_header] = df[x_header].fillna(0)
    if give_category_names:
      bin_labels = ["1_no_foll_investors","2_low_num_foll_investors", "3_medium_num_foll_investors", "4_high_num_foll_investors"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#
      
  if x_header == "num_unique_rounds":
    #decide if the final categories should be named or numerical categories:
    df[x_header] = df[x_header].fillna(0)
    if give_category_names:
      bin_labels = ["1_no_foll_investors","2_low_num_foll_investors", "3_medium_num_foll_investors", "4_high_num_foll_investors"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "fnf_degree_mean":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    cut_info = ['Slicing info: Qartile cut']

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut'] 
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "fnf_degree_delta_survey":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#
  
  if x_header == "fnf_degree_rel_difference":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "friends_degree_mean":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "friends_degree_delta_survey":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut": 
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "friends_degree_rel_difference":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "followers_degree_mean":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut'] 
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "followers_degree_delta_survey":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "followers_degree_rel_difference":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "unweighted_bi_direct_degree_mean":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "unweighted_bi_direct_degree_delta_survey":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "unweighted_bi_direct_degree_rel_difference":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Half split and split halves again'] 
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "fr_weighted_bi_direct_degree_mean":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "fr_weighted_bi_direct_degree_delta_survey":  
  #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
     
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "fr_weighted_bi_direct_degree_rel_difference":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut": 
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "followers_div_by_friends_degree_ratio": 
    if fill_nan:
      df[x_header] = df[x_header].fillna(-1)
    #decide if the final categories should be named or numerical categories:
    bin_labels = ["X1_NA", "X2_No_friends", "1_low_follow_on", "2_medium_follow_on", "3_high_follow_on"]
    #define how we want to cut
    print(df[[x_header]].isnull().sum())
    cut_type = "cut" #cuts with a self defined list
    
    if cut_type == "cut": 
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = [-2]+zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "c_and_i_betweenness_centrality":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
  
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut": 
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "c_and_i_eigenvector_centrality":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "followers_max_10_matches":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = cut_info = ['Slicing info: Self defined thresholds'] 
      #document cutting:
      bin_values = [-0.001,0,1,2,10]
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "followers_max_100_matches":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "followers_max_1000_matches":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "followers_max_10000_matches":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "friends_max_10_matches":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = cut_info = ['Slicing info: Self defined thresholds'] 
      #document cutting:
      bin_values = [-0.001,0,1,2,10]
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "friends_max_100_matches":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "friends_max_1000_matches":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "friends_max_10000_matches":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "bi_direct_max_10_matches":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = cut_info = ['Slicing info: Self defined thresholds'] 
      #document cutting:
      bin_values = [-0.001,0,1,2,10]
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "bi_direct_max_100_matches":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "bi_direct_max_1000_matches":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "bi_direct_max_10000_matches":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals 
  #-----------------------------------------------------------------------------------------------------------------#
  
  if x_header == "ego_followers_density":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = cut_info = ['Slicing info: Self defined thresholds'] 
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      print(bin_values)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      print(zip_variable_intervals)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "ego_friends_density":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = cut_info = ['Slicing info: Self defined thresholds'] 
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      print(bin_values)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      print(zip_variable_intervals)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "ego_bi_direct_density":  
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_none","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = cut_info = ['Slicing info: Self defined thresholds'] 
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      print(bin_values)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      print(zip_variable_intervals)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "followers_match_count_mean":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "followers_match_count_delta_survey":  
  #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
     
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values = half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "followers_match_count_rel_difference":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut": 
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#
  
  if x_header == "followers_match_count_per_user":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#
  
  if x_header == "followers_match_w_count_mean":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#
  
  if x_header == "followers_match_w_count_per_user_avg":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "followers_match_w_count_delta_survey":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut": 
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "followers_match_w_count_rel_difference":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut": 
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "friends_match_count_mean":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "friends_match_count_delta_survey":  
  #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values = half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "friends_match_count_rel_difference":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut": 
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "friends_match_count_per_user":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "friends_match_w_count_mean":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "friends_match_w_count_per_user_avg":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "friends_match_w_count_delta_survey":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut": 
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "friends_match_w_count_rel_difference":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut": 
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "unweighted_bi_direct_match_count_mean":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "unweighted_bi_direct_match_count_delta_survey":  
  #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values = half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "unweighted_bi_direct_match_count_rel_difference":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut": 
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "unweighted_bi_direct_match_count_per_user":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "unweighted_bi_direct_match_w_count_mean":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "unweighted_bi_direct_match_w_per_user_count_avg":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_very_low","2_low","3_high", "4_very_high"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets
    
    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut":
      cut_info = ['Slicing info: Cut off the "0" and split the rest in third cut']
      #document cutting:
      bin_values = zero_thirds(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "unweighted_bi_direct_match_w_count_delta_survey":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined list
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut": 
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

  if x_header == "unweighted_bi_direct_match_w_count_rel_difference":
    #decide if the final categories should be named or numerical categories:
    if give_category_names:
      bin_labels = ["1_high_loss","2_small_loss", "3_true_zero", "4_small_gain", "5_high_gain"]
    if not give_category_names:
      bin_labels = quattro
    #define how we want to cut
    cut_type = "cut" #cuts with a self defined listl
    # cut_type = "qcut"  #cuts into q=n n different buckets

    if cut_type == "qcut": 
      zip_variable_intervals = interval_documentor(x_header,pd.qcut(df[x_header], q=4),bin_names=bin_labels)
      #cut and overwrite:
      df[x_header] = pd.qcut(df[x_header], q=len(bin_labels), labels = bin_labels)
    if cut_type == "cut": 
      cut_info = ['Slicing info: Half split and split halves again']
      #document cutting:
      bin_values =  half_split(df,x_header)
      zip_variable_intervals = interval_documentor(x_header,pd.cut(df[x_header], bins=bin_values),bin_names=bin_labels)
      #cut and overwrite:
      if fill_nan:
        df[x_header] = df[x_header].fillna(0)
      df[x_header] = pd.cut(df[x_header], bins=bin_values, labels = bin_labels)
    all_variable_interval_description_dict[x_header] = cut_info + zip_variable_intervals
  #-----------------------------------------------------------------------------------------------------------------#

    print("number of nans:", df[x_header].isnull().sum(axis = 0))
    print(df[x_header].describe())
    # print(df[x_header].quantile([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]))
    print(df[x_header].value_counts())
    print(counter, "of", len(df.columns))
    # pp.pprint(all_variable_interval_description_dict)
df.iloc[:10,20:]

In [ ]:
import json
file_name = "/ML_categories.json"

with open(base_path + file_name, 'w', encoding='utf-8') as f:
    json.dump(all_variable_interval_description_dict, f, ensure_ascii=False, indent=4)

# Post-process

In [ ]:
nan_cols = [i for i in df.columns if df[i].isnull().any()]
if len(nan_cols)>0:
  print("These columns have 'nan' values:")
  df[nan_cols].isnull().sum()

  # Can be used to replace nan values based on other column
  # df["followers_match_w_count_delta_survey"] = df.followers_match_w_count_delta_survey[df['has_twitter'] != 0].fillna(0)
else:
  print("There are NO 'nan' values in ANY column!")

make columns category

In [ ]:
df['has_twitter'] = df.has_twitter.astype('category')
df['has_website'] = df.has_website.astype('category')
df['b2b'] = df.b2b.astype('category')
df['had_event'] = df.had_event.astype('category')
df['Angel'] = df.Angel.astype('category')
df['Grant'] = df.Grant.astype('category')
df['Crowdfunding'] = df.Crowdfunding.astype('category')
df['Seed'] = df.Seed.astype('category')
df['Series_A'] = df.Series_A.astype('category')
df['Series_B'] = df.Series_B.astype('category')
df['hq_city_ecosystem'] = df.hq_city_ecosystem.astype('category')
df['accelerated'] = df.accelerated.astype('category')
df['invested'] = df.invested.astype('category')

In [ ]:
df = df.replace(False, "No")
df = df.replace(True, "Yes")

Decide which variables to test against target

In [ ]:
selected_target_candidates = ['dt_avg',
                              'num_sum_raised_to_date',
                              'max_post_money_valuation',
                              'had_event',
                              'Angel',
                              'Grant',
                              'Crowdfunding',
                              'Seed',
                              'Series_A',
                              'Series_B',
                              'accelerated',
                              'invested',
                              'num_rounds',
                              'num_unique_rounds',
                              'c_and_i_betweenness_centrality',
                              'c_and_i_eigenvector_centrality',
                              'num_sum_investors',
                              'num_sum_follow_on_investors',
                              'new_to_followon_investor_ratio']

definite_variables = [h for h in list(df.columns) if h not in selected_target_candidates]
# pp.pprint(definite_variables)

selected_target = 'dt_avg'
# selected_target = 'num_sum_raised_to_date'
# selected_target = 'max_post_money_valuation'
# selected_target = 'had_event'
# selected_target = 'Angel'
# selected_target = 'Grant'
# selected_target = 'Crowdfunding'
# selected_target = 'Seed'
# selected_target = 'Series_A'
# selected_target = 'Series_B'
# selected_target = 'accelerated'
# selected_target = 'invested'
# selected_target = 'num_rounds'
# selected_target = 'num_unique_rounds'
# selected_target = 'c_and_i_betweenness_centrality'
# selected_target = 'c_and_i_eigenvector_centrality'
# selected_target = 'num_sum_investors'
# selected_target = 'num_sum_follow_on_investors'
# selected_target = 'new_to_followon_investor_ratio'

if selected_target == "dt_avg":
  print("Target is:", selected_target)
  # put_back_in = ['accelerated','c_and_i_betweenness_centrality','c_and_i_eigenvector_centrality','num_sum_investors','num_sum_follow_on_investors','new_to_followon_investor_ratio']
  put_back_in = list(set(['accelerated','new_to_followon_investor_ratio']).difference(set([selected_target])))
  variables_based_on_target = definite_variables + put_back_in
  print("Of reinserted variables of the targets,", put_back_in)
  # to restore the original order:
  selected_variables = [h for h in list(df.columns) if h in variables_based_on_target]
  # print(selected_variables)
  df_variables = df[selected_variables].copy()
  df_targets = df[[selected_target]].copy()

if selected_target == "num_sum_raised_to_date":
  print("Target is:", selected_target)
  put_back_in = list(set(['accelerated','c_and_i_betweenness_centrality','c_and_i_eigenvector_centrality','num_sum_investors','new_to_followon_investor_ratio','had_event', 'accelerated']).difference(set([selected_target])))
  variables_based_on_target = definite_variables + put_back_in
  print("Of reinserted variables of the targets,", put_back_in)
  # to restore the original order:
  selected_variables = [h for h in list(df.columns) if h in variables_based_on_target]
  # print(selected_variables)
  df_variables = df[selected_variables].copy()
  df_targets = df[[selected_target]].copy()

if selected_target == "max_post_money_valuation":
  print("Target is:", selected_target)
  put_back_in = list(set(['accelerated','c_and_i_betweenness_centrality','c_and_i_eigenvector_centrality','num_sum_investors','new_to_followon_investor_ratio','had_event', 'accelerated']).difference(set([selected_target])))
  variables_based_on_target = definite_variables + put_back_in
  print("Of reinserted variables of the targets,", put_back_in)
  # to restore the original order:
  selected_variables = [h for h in list(df.columns) if h in variables_based_on_target]
  # print(selected_variables)
  df_variables = df[selected_variables].copy()
  df_targets = df[[selected_target]].copy()

if selected_target not in ['dt_avg', 'num_sum_raised_to_date', 'max_post_money_valuation']:
  print("Target is:", selected_target)
  put_back_in = list(set(['accelerated','c_and_i_betweenness_centrality','c_and_i_eigenvector_centrality','num_sum_investors','new_to_followon_investor_ratio','had_event','accelerated']).difference(set([selected_target])))
  variables_based_on_target = definite_variables + put_back_in
  print("Of reinserted variables of the targets,",put_back_in)
  # to restore the original order:
  selected_variables = [h for h in list(df.columns) if h in variables_based_on_target]
  # print(selected_variables)
  df_variables = df[selected_variables].copy()
  df_targets = df[[selected_target]].copy()

# Show variables that are out:
not_in_headers = set(df.columns).difference(set(selected_variables)) 
# print("Variables not in:", not_in_headers)

# Check target not in variables:
# if selected_target in df_variables.columns:
  # print("WARNING, target variable in explaining variables")

In [ ]:
# df_variables.head() 

In [ ]:
# df_targets.head()

# Cramer

In [ ]:
cramer_val_and_p_v_relevant_headers = cramer_df_selector(df_variables,df_targets,save=True)
# pp.pprint(cramer_val_and_p_v_relevant_headers)

# ML

In [ ]:
val_and_p_v_relevant_headers_temp = selected_variables
# val_and_p_v_relevant_headers_temp = cramer_val_and_p_v_relevant_headers

# val_and_p_v_relevant_headers_temp = [
#                                 'inbound_links',
#                                 'global_pagerank',
#                                 'b2b',
#                                 # 'num_sum_investors',
#                                 # 'num_sum_follow_on_investors',
#                                 'accelerated',
#                                 # 'invested',
#                                 # 'new_to_followon_investor_ratio',
#                                 'fnf_degree_delta_survey',
#                                 'fnf_degree_rel_difference',
#                                 'friends_degree_delta_survey',
#                                 'friends_degree_rel_difference',
#                                 'followers_degree_mean',
#                                 'followers_degree_delta_survey',
#                                 'followers_degree_rel_difference',
#                                 'unweighted_bi_direct_degree_delta_survey',
#                                 'unweighted_bi_direct_degree_rel_difference',
#                                 'fr_weighted_bi_direct_degree_delta_survey',
#                                 'followers_div_by_friends_degree_ratio',
#                                 # 'c_and_i_betweenness_centrality',
#                                 # 'c_and_i_eigenvector_centrality',
#                                 'followers_max_1000_matches',
#                                 'followers_max_10000_matches',
#                                 'friends_max_100_matches',
#                                 'bi_direct_max_1000_matches',
#                                 'bi_direct_max_10000_matches',
#                                 'followers_match_count_mean',
#                                 'followers_match_count_delta_survey',
#                                 'followers_match_count_rel_difference',
#                                 'followers_match_w_count_mean',
#                                 'followers_match_w_count_delta_survey',
#                                 'followers_match_w_count_rel_difference',
#                                 'friends_match_count_delta_survey',
#                                 'friends_match_count_rel_difference',
#                                 'friends_match_w_count_delta_survey',
#                                 'friends_match_w_count_rel_difference',
#                                 'unweighted_bi_direct_match_count_delta_survey',
#                                 'unweighted_bi_direct_match_count_rel_difference',
#                                 'unweighted_bi_direct_match_w_count_delta_survey',
#                                 'unweighted_bi_direct_match_w_count_rel_difference'
#                                 ]
# print(len(val_and_p_v_relevant_headers_temp))

# print(val_and_p_v_relevant_headers_temp)
# print(selected_target)
# ML(df,ML_selected_variables = val_and_p_v_relevant_headers_temp, ML_selected_target = selected_target, no_iterations=10)

#headers not in:
not_in_headers = set(df.columns).difference(set(selected_variables)) 

In [ ]:
print("not in list", len(not_in_headers), "elements:", not_in_headers)
print("list", len(val_and_p_v_relevant_headers_temp), "elements:", val_and_p_v_relevant_headers_temp)
factorized_nw = df[val_and_p_v_relevant_headers_temp].copy()
for column in factorized_nw.columns.values:
    f, _ = pd.factorize(factorized_nw[column])
    factorized_nw.loc[:,column] = f
ohe = sp.OneHotEncoder()
X = factorized_nw
# unique_labels = list(df.dt_avg.unique())
unique_labels = list(df[selected_target].unique())
# y = [unique_labels.index(i) for i in df.dt_avg.values]
y = [unique_labels.index(i) for i in df[selected_target].values]

ohe.fit(X)
X_plot = X.copy()
#X = ohe.transform(X).toarray()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("selected_target:", selected_target)
for i in range(1,10):
    tree = DecisionTreeClassifier(max_depth=i, random_state=42)
    tree.fit(X_train,y_train)
    y_pred = tree.predict(X_test)
    
    print("Max depth: {} - accuracy:".format(i), accuracy_score(y_test, y_pred, normalize=True))
    print()

print(classification_report(y_test, y_pred, target_names=unique_labels))

In [ ]:
forest = RandomForestClassifier(max_depth=2, random_state=42)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

In [ ]:
print("not in list", len(not_in_headers), "elements:", not_in_headers)
print("list", len(val_and_p_v_relevant_headers_temp), "elements:", val_and_p_v_relevant_headers_temp)

print(classification_report(y_test, y_pred, target_names=unique_labels))

In [ ]:
params = {
          "objective" : "multiclass",
          "num_class" : len(unique_labels),
          "num_leaves" : 60,
          "max_depth": -1,
          "learning_rate" : 0.01,
          "bagging_fraction" : 0.9,  # subsample
          "feature_fraction" : 0.9,  # colsample_bytree
          "bagging_freq" : 5,        # subsample_freq
          "bagging_seed" : 2018,
          "verbosity" : -1
          }

In [ ]:
print(unique_labels)

print(len(X_train))

splitsize_val= round(0.7*len(X_train))
print(splitsize_val)

In [ ]:
import lightgbm as lgb

splitsize_val= round(0.7*len(X_train))
print(splitsize_val)
lgtrain, lgval = lgb.Dataset(X_train[0:splitsize_val], y_train[0:splitsize_val]), lgb.Dataset(X_train[splitsize_val:], y_train[splitsize_val:])
# lgbmodel = lgb.train(params, lgtrain, 2000, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=200)
lgbmodel = lgb.train(params, lgtrain, 2000, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=100)

In [ ]:
y_pred = lgbmodel.predict(X_test)
y_pred = [np.argmax(pred) for pred in y_pred] 

In [ ]:
print("not in list", len(not_in_headers), "elements:", not_in_headers)
print("list", len(val_and_p_v_relevant_headers_temp), "elements:", val_and_p_v_relevant_headers_temp)

print("Max depth: {} - accuracy:".format(i), accuracy_score(y_test, y_pred, normalize=True))

print(classification_report(y_test, y_pred,target_names=unique_labels))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp_split = pd.DataFrame(sorted(zip(lgbmodel.feature_importance(importance_type='split'), X_plot.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp_split.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()
# plt.savefig('lgbm_importances-01.png')

In [ ]:
feature_imp_gain = pd.DataFrame(sorted(zip(lgbmodel.feature_importance(importance_type='gain'), X_plot.columns)), columns=['Value','Feature'])
plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp_gain.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()

In [ ]:
df_feature_importance_split = feature_imp_split.sort_values(by="Value", ascending=False)
df_feature_importance_split["norm_feature_imp"] = df_feature_importance_split[['Value']].mul(100).div(df_feature_importance_split[['Value']].max()).round(1)

df_feature_importance_gain = feature_imp_gain.sort_values(by="Value", ascending=False)
df_feature_importance_gain["norm_feature_imp"] = df_feature_importance_gain[['Value']].mul(100).div(df_feature_importance_gain[['Value']].max()).round(1)

# save to excel
with pd.ExcelWriter(os.path.join(base_path, "feature-importance/" + selected_target + '_feature_importance.xlsx')) as writer:
  df_feature_importance_split.to_excel(writer, sheet_name= "split" , index=True)
  df_feature_importance_gain.to_excel(writer, sheet_name= "gain" , index=True)

In [ ]:
df_feature_importance_split

In [ ]:
df_feature_importance_gain